# 預處理

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [2]:
import re
def rm_tags(text):
    re_tag = re.compile(r'<[^>]+>')
    return re_tag.sub('', text)

In [5]:
import os
def read_files(filetype):
    path = '../datasets/aclImdb/'
    file_list = []
    positive_path = path + filetype + '/pos/'
    for f in os.listdir(positive_path):
        file_list += [positive_path + f]
    negative_path = path + filetype + '/neg/'
    for f in os .listdir(negative_path):
        file_list += [negative_path + f]
    print('read ', filetype, ' files: ', len(file_list))    
    all_labels = ([1] * 12500 + [0] * 12500)
    all_texts = []
    for fi in file_list:
        with open(fi, encoding = 'utf8',) as file_input:
            all_texts += [rm_tags(' '.join(file_input.readlines()))]
    return all_labels, all_texts

In [6]:
y_train, train_text = read_files('train')

read  train  files:  25000


In [7]:
y_test, test_text = read_files('test')

read  test  files:  25000


In [8]:
# y_train[0]

In [9]:
# train_text[0]

In [10]:
# y_test[0]

In [11]:
# test_text[0]

### Token

In [ ]:
token = Tokenizer(num_words = 2000)
token.fit_on_texts(train_text)
print(token.document_count)

25000


### Text to List

In [ ]:
x_train_seq = token.texts_to_sequences(train_text)
x_test_seq = token.texts_to_sequences(test_text)

In [ ]:
# print(train_text[0])

In [ ]:
# print(x_train_seq[0])

In [ ]:
x_train = sequence.pad_sequences(x_train_seq, maxlen = 100)
x_test = sequence.pad_sequences(x_train_seq, maxlen = 100)

In [ ]:
# print(x_train_seq[0])

In [ ]:
# print(len(x_train_seq[0]))

In [ ]:
# print(x_train[0])

In [ ]:
# print(len(x_train[0]))